In [7]:
import math
import torch

# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64   # use double precision

from torch.optim import Adam

from botorch.models import SingleTaskGP
from botorch.acquisition import LogExpectedImprovement  # safer than EI
from botorch.test_functions.synthetic import AckleyMixed
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.constraints import GreaterThan

In [8]:
# AckleyMixed benchmark
dim = 10
func = AckleyMixed(dim=dim, negate=True)  # negate=True => maximization


In [9]:
def generate_mixed_candidates(n, dim):
    X = torch.empty(n, dim, dtype=dtype, device=device)
    # first dim-3 are binary
    X[:, :dim-3] = torch.randint(0, 2, (n, dim-3), dtype=dtype, device=device)
    # last 3 are continuous [0,1]
    X[:, dim-3:] = torch.rand(n, 3, dtype=dtype, device=device)
    return X

def generate_initial_data(n=8):
    X = generate_mixed_candidates(n, dim)
    Y = func(X).unsqueeze(-1)
    return X, Y

train_X, train_Y = generate_initial_data()
print("Initial best value:", train_Y.max().item())


Initial best value: -2.728720250292437


In [10]:
model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))


In [11]:
mll = ExactMarginalLogLikelihood(model.likelihood, model).to(train_X)


In [12]:
optimizer = Adam(model.parameters(), lr=0.05)


In [13]:
NUM_EPOCHS = 150

model.train()
for epoch in range(NUM_EPOCHS):
    optimizer.zero_grad()
    output = model(train_X)
    loss = -mll(output, model.train_targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.3f}")


Epoch 10/150 - Loss: 5.230
Epoch 20/150 - Loss: 4.985
Epoch 30/150 - Loss: 4.763
Epoch 40/150 - Loss: 4.573
Epoch 50/150 - Loss: 4.420
Epoch 60/150 - Loss: 4.299
Epoch 70/150 - Loss: 4.207
Epoch 80/150 - Loss: 4.138
Epoch 90/150 - Loss: 4.088
Epoch 100/150 - Loss: 4.052
Epoch 110/150 - Loss: 4.027
Epoch 120/150 - Loss: 4.010
Epoch 130/150 - Loss: 3.999
Epoch 140/150 - Loss: 3.993
Epoch 150/150 - Loss: 3.988


In [14]:
N_BATCH = 5

for i in range(N_BATCH):
    # Re-fit GP on updated data
    model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
    model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))
    mll = ExactMarginalLogLikelihood(model.likelihood, model).to(train_X)
    optimizer = Adam(model.parameters(), lr=0.05)

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(train_X)
        loss = -mll(output, model.train_targets)
        loss.backward()
        optimizer.step()

    model.eval()

    # Acquisition function
    acq = LogExpectedImprovement(model=model, best_f=train_Y.max())

    # Sample candidate mixed points
    candidates = generate_mixed_candidates(n=2000, dim=dim)
    acq_values = acq(candidates.unsqueeze(1))
    new_x = candidates[acq_values.argmax()]

    # Evaluate and add to dataset
    new_y = func(new_x.unsqueeze(0)).unsqueeze(-1)
    train_X = torch.cat([train_X, new_x.unsqueeze(0)], dim=0)
    train_Y = torch.cat([train_Y, new_y], dim=0)

    print(f"Iteration {i+1} | Best value so far: {train_Y.max().item():.4f}")


Iteration 1 | Best value so far: -2.1728
Iteration 2 | Best value so far: -2.1728
Iteration 3 | Best value so far: -1.7282
Iteration 4 | Best value so far: -1.7282
Iteration 5 | Best value so far: -1.7282
